# Lab 3

In [24]:
import numpy as np
import pandas as pd

In [27]:
X = np.load('./data/data.npy')
df = pd.DataFrame.from_dict(X.flat[0])

In [26]:
df

,ic50,smiles,target
0,11600,Nc1nc(N)c2c(CSc3ccc(Cl)c(Cl)c3)c[nH]c2n1,CHEMBL664913
1,9000,NC(=S)\N=N\c1c(O)[nH]c2ccc(I)cc12,CHEMBL812106
2,17900,[Cl-].C[N+](C)(CCCN1c2ccccc2Sc3ccc(Cl)cc13)Cc4...,CHEMBL811877
3,4200,COc1cc(OC)cc(c1)N(C)Cc2c[nH]c3nc(N)nc(O)c23,CHEMBL664920
4,2700,COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC,CHEMBL664913
5,83,Cc1c(CN(C=O)c2ccc(Cl)c(Cl)c2)cnc3nc(N)nc(N)c13,CHEMBL665696
6,15000,Cc1ccc2[nH]c(O)c(\N=N\C(=N)S)c2c1,CHEMBL805131
7,20,CN(Cc1ccc2nc(N)nc(N)c2n1)c3cccc(Cl)c3,CHEMBL665702
8,20,CCCc1cc(O)c(Oc2ccc(cc2Cl)C#N)c(O)c1,CHEMBL1246345
9,62,Nc1nc(N)c2CC(Cc3cccs3)CCc2n1,CHEMBL665602
